### Imports

In [1]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 186 µs (started: 2023-06-09 09:41:36 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    
    subgroups_id_op = {}
    for group in groups_id:
        subgroups_id_op[group] = [option]
else:
    subgroups_id_op = subgroups_id
print("Subgroups id op:", subgroups_id_op)



Exp:		 exp5
Method:		 dgi
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
Option:		 
Subgroups id op: {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
time: 2.93 ms (started: 2023-06-09 09:41:36 -05:00)


### Edge embeddings

In [3]:
# get edges embeddings

edge_embeddings_global(exp, method, groups_id, subgroups_id_op)

100%|██████████| 3/3 [51:06<00:00, 1022.03s/it]

time: 51min 6s (started: 2023-06-09 09:41:36 -05:00)


In [4]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, groups_id[0], subgroups_id_op[groups_id[0]][0]), index_col=[0, 1])
df_edge_embeddings.head()

0             1             2
0 1  1.141083e-03  2.847290e-09  1.935469e-03
  2  5.719141e-09  1.840410e-11  1.119957e-08
  3  1.711179e-03  4.984949e-08  2.932494e-03
  4  1.806796e-04  1.005588e-07  3.249768e-04
  5  3.780943e-04  5.812921e-10  6.364315e-04

time: 491 ms (started: 2023-06-09 10:32:42 -05:00)


### Concat edge embeddings

In [5]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    k = 0
    for subgroup in tqdm(subgroups_id_op[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
        k += 1
    
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True)

100%|██████████| 3/3 [03:13<00:00, 64.53s/it]

time: 3min 13s (started: 2023-06-09 10:32:43 -05:00)


In [6]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])
df_edge_embeddings_concat.head()

0             1             2  subgroup
0 1  1.141083e-03  2.847290e-09  1.935469e-03         0
  2  5.719141e-09  1.840410e-11  1.119957e-08         0
  3  1.711179e-03  4.984949e-08  2.932494e-03         0
  4  1.806796e-04  1.005588e-07  3.249768e-04         0
  5  3.780943e-04  5.812921e-10  6.364315e-04         0

time: 5.06 s (started: 2023-06-09 10:35:57 -05:00)


In [7]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

100%|██████████| 3/3 [14:29<00:00, 289.79s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 14min 29s (started: 2023-06-09 10:36:02 -05:00)


### Outliers detection

In [8]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 1.99 ms (started: 2023-06-09 10:50:31 -05:00)


In [9]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [01:04<00:00, 21.37s/it]

time: 1min 4s (started: 2023-06-09 10:50:31 -05:00)


In [10]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
0 3    0.001711  4.984949e-08  0.002932         0
  8    0.006827  3.970386e-07  0.011770         0
  9    0.005209  4.986619e-07  0.009032         0
  10   0.018792  6.724997e-06  0.033386         0
1 131  0.005130  3.046072e-06  0.009246         0

time: 6.41 ms (started: 2023-06-09 10:51:35 -05:00)


In [11]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

  0%|          | 0/3 [00:00<?, ?it/s]

Total: 12232238
Num. of inliers: 11009014
Num. of inliers: 1223224


 33%|███▎      | 1/3 [04:45<09:30, 285.16s/it]

Total: 17089033
Num. of inliers: 15380129
Num. of inliers: 1708904


 67%|██████▋   | 2/3 [11:33<05:57, 357.72s/it]

...

100%|██████████| 3/3 [13:54<00:00, 278.28s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 13min 54s (started: 2023-06-09 10:51:36 -05:00)


###  Filter common edges

In [12]:
print(subgroups_id)
print(subgroups_id_op)


{'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
{'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
time: 382 µs (started: 2023-06-09 11:05:30 -05:00)


In [13]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id_op[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"id": "string"})
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []
    
    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[subgroups_id_op[group][row[-1]]]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)

11009014it [05:15, 34925.71it/s]it/s]
15380129it [07:15, 35277.10it/s]8, 324.31s/it]
5439862it [02:36, 34752.83it/s]37, 397.16s/it]
100%|██████████| 3/3 [15:33<00:00, 311.15s/it]

time: 15min 33s (started: 2023-06-09 11:05:31 -05:00)


In [14]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
59.0049  369.1213  0.001711  4.984949e-08  0.002932         0
         641.1223  0.006827  3.970386e-07  0.011770         0
         650.2145  0.005209  4.986619e-07  0.009032         0
         977.8237  0.018792  6.724997e-06  0.033386         0
274.0123 191.046   0.005130  3.046072e-06  0.009246         0

time: 11 ms (started: 2023-06-09 11:21:04 -05:00)


In [15]:
# format id
if option:
    for group in tqdm(groups_id):
        # format
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        list_index = []

        for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
            list_index.append((row[0][0][1:], row[0][1][1:]))
        
        # set new index
        df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
        df_edge_embeddings_concat_filter

time: 1.3 ms (started: 2023-06-09 11:21:04 -05:00)


In [16]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
59.0049  369.1213  0.001711  4.984949e-08  0.002932         0
         641.1223  0.006827  3.970386e-07  0.011770         0
         650.2145  0.005209  4.986619e-07  0.009032         0
         977.8237  0.018792  6.724997e-06  0.033386         0
274.0123 191.046   0.005130  3.046072e-06  0.009246         0

time: 13.8 ms (started: 2023-06-09 11:21:04 -05:00)


In [17]:
# filter diferente edges

if option:
    for group in tqdm(groups_id):
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter.index.get_level_values(0) != df_edge_embeddings_concat_filter.index.get_level_values(1)]
        dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

time: 421 µs (started: 2023-06-09 11:21:04 -05:00)


In [18]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
print(df_edge_embeddings_concat_filter.shape)
df_edge_embeddings_concat_filter.head()

(11009014, 4)


0             1         2  subgroup
59.0049  369.1213  0.001711  4.984949e-08  0.002932         0
         641.1223  0.006827  3.970386e-07  0.011770         0
         650.2145  0.005209  4.986619e-07  0.009032         0
         977.8237  0.018792  6.724997e-06  0.033386         0
274.0123 191.046   0.005130  3.046072e-06  0.009246         0

time: 12.4 ms (started: 2023-06-09 11:21:04 -05:00)


In [19]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # count
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["level_0", "level_1"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]

    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:15<00:00,  5.07s/it]

time: 15.2 s (started: 2023-06-09 11:21:04 -05:00)


In [20]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,333.0598,345.0355
1,714.5447,820.5967
2,139.0232,178.9792
3,341.177,343.1146
4,139.0232,179.0031


time: 9.89 ms (started: 2023-06-09 11:21:20 -05:00)


In [21]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23454 entries, 0 to 23453
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  23454 non-null  object
 1   target  23454 non-null  object
dtypes: object(2)
memory usage: 549.7+ KB
time: 10.1 ms (started: 2023-06-09 11:21:20 -05:00)


In [22]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

100%|██████████| 3/3 [00:00<00:00, 41.63it/s]

time: 75.3 ms (started: 2023-06-09 11:21:20 -05:00)


In [23]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,333.0598,345.0355
1,714.5447,820.5967
2,139.0232,178.9792
3,341.177,343.1146
4,139.0232,179.0031


time: 9.04 ms (started: 2023-06-09 11:21:20 -05:00)


In [24]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23454 entries, 0 to 23453
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  23454 non-null  string
 1   target  23454 non-null  string
dtypes: string(2)
memory usage: 549.7 KB
time: 13.7 ms (started: 2023-06-09 11:21:20 -05:00)


In [25]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

100%|██████████| 3/3 [01:11<00:00, 23.69s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
20620,109.0407,114.056,0.684680,0.654750,0.527725,0.859966,0.823646
20757,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
20680,109.0407,131.0462,0.802158,0.721020,0.613603,0.872517,0.867231
20677,109.0407,131.0824,0.808212,0.663848,0.719276,0.857543,0.913011
20703,109.0407,132.086,0.764665,0.614573,0.603960,0.893537,0.878882


time: 1min 11s (started: 2023-06-09 11:21:20 -05:00)


In [26]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
20620,109.0407,114.056,0.684680,0.654750,0.527725,0.859966,0.823646
20757,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
20680,109.0407,131.0462,0.802158,0.721020,0.613603,0.872517,0.867231
20677,109.0407,131.0824,0.808212,0.663848,0.719276,0.857543,0.913011
20703,109.0407,132.086,0.764665,0.614573,0.603960,0.893537,0.878882


time: 14.2 ms (started: 2023-06-09 11:22:32 -05:00)


### Filter by STD and average weight

In [27]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

100%|██████████| 3/3 [00:14<00:00,  4.71s/it]


,source,target,weight
0,109.0407,114.056,0.710154
1,109.0407,127.0513,0.830812
2,109.0407,131.0462,0.775306
3,109.0407,131.0824,0.792378
4,109.0407,132.086,0.751123


time: 14.1 s (started: 2023-06-09 11:22:32 -05:00)


In [28]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, groups_id[0], option),
                              dtype={"source": "string", "target": "string"})
df_common_edges.head()

,source,target,weight
0,109.0407,114.056,0.710154
1,109.0407,127.0513,0.830812
2,109.0407,131.0462,0.775306
3,109.0407,131.0824,0.792378
4,109.0407,132.086,0.751123


time: 23.8 ms (started: 2023-06-09 11:22:46 -05:00)


In [29]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

 33%|███▎      | 1/3 [00:00<00:00,  2.27it/s]

Group: WT
Num. nodes: 1916
Num. edges: 23328



 67%|██████▋   | 2/3 [00:01<00:00,  1.25it/s]

Group: zwf1^
Num. nodes: 4368
Num. edges: 521963



100%|██████████| 3/3 [00:02<00:00,  1.24it/s]

Group: pck1^
Num. nodes: 5046
Num. edges: 454548

time: 2.41 s (started: 2023-06-09 11:22:46 -05:00)
